In [1]:
from datetime import datetime
import pandas as pd
import warnings
import re
warnings.simplefilter('ignore')
import numpy as np

In [2]:
locationsDF = pd.read_csv('worldcitiespop.csv',index_col=None, header=0, lineterminator='\n')

In [3]:
incomeDF = pd.read_csv('kaggle_income.csv',index_col=None, header=0, lineterminator='\n', encoding='latin-1', usecols=['City', 'Mean'])

In [4]:
stateIncomeDF = pd.read_csv('incomeByState.csv',index_col=None, header=0, lineterminator='\n', encoding='latin-1')

In [5]:
stateDF = pd.read_csv('uscitiesv1.4.csv',index_col=None, header=0, lineterminator='\n', encoding='latin-1')

In [6]:
import csv 
with open('incomeByState.csv', mode='r') as infile:
    reader = csv.reader(infile)
    stateIncomeDict = {columns[0].lower():columns[1] for columns in reader}
with open('countryCodeIncome.csv', mode='r') as infile:
    reader = csv.reader(infile)
    countryIncomeDict = {columns[0].lower():columns[1] for columns in reader}
with open('countryCodeEducation.csv', mode='r') as infile:
    reader = csv.reader(infile)
    countryEducationDict = {columns[0].lower():columns[1] for columns in reader}
with open('stateEducation.csv', mode='r') as infile:
    reader = csv.reader(infile)
    stateEducationDict = {columns[0].lower():columns[1] for columns in reader}

In [7]:
countriesFile = open('countries.txt', 'r')
countriesList = [line.lower().strip() for line in countriesFile.readlines()]

In [8]:
statesFile = open('states.txt', 'r')
statesList = [line.lower().strip() for line in statesFile.readlines()]

In [9]:
stateDict = dict()
for index,row in stateDF.iterrows():
    stateDict[row['city'].lower()] = row['state_name'].lower()

In [10]:
def removeNums(text):
    splitText = text.split()
    output = []
    for item in splitText:
        if item.isalpha():
            output.append(item)
    if len(output) > 0:
        return ' '.join(output)
    else:
        return ''

In [11]:
cityDict = dict()
for index,row in locationsDF.iterrows():
    cityDict[str(row['City']).lower()] = str(row['Country']).lower()


In [12]:
import unicodedata
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [13]:
%%time
#strip links
import re
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 8.82 µs


In [14]:
countDict = dict()
incomeDF['City'] = incomeDF['City'].apply(lambda x: x.lower())
countDict = incomeDF['City'].value_counts()

In [15]:
incomeDict = dict()
for index, row in incomeDF.iterrows():
    if row['Mean'] == 0:
        countDict[row['Mean']]-=1
    else: 
        if incomeDict.get(row['City']) is None:
            incomeDict[row['City']] = 0
        incomeDict[row['City']] = incomeDict.get(row['City']) + row['Mean']/countDict.get(row['City'])

In [16]:
def extractCity(text):
    if text == '':
        return ''
    splitText = text.split()
    if len(splitText) > 1:
        firstTwoWords = splitText[0] + ' ' + splitText[1]
        if firstTwoWords in incomeDict:
            return firstTwoWords
        lastTwoWords = splitText[len(splitText)-1] + ' ' + splitText[len(splitText)-2]
        if lastTwoWords in incomeDict:
            return lastTwoWords
        if len(splitText) > 2:
            firstThreeWords = splitText[0] + ' ' + splitText[1]+ ' ' + splitText[2]
            lastThreeWords = splitText[len(splitText)-1] + ' ' + splitText[len(splitText)-2]+ ' ' + splitText[len(splitText)-3]
            if firstThreeWords in incomeDict:
                return firstThreeWords
            if lastThreeWords in incomeDict:
                return lastThreeWords
        for i in range(0, len(splitText)):
            if splitText[i] in incomeDict:
                if splitText[i] != 'north' and splitText[i] != 'west' and splitText[i] != 'east' and splitText[i] !='south' and splitText[i] !='central' and splitText[i] !='york':
                    return splitText[i]
    else:
        if text in incomeDict:
            if text != 'north' and text != 'west' and text != 'east' and text !='south':
                return text
    return ''

In [17]:
def extractState(text):
    if text == '':
        return ''
    splitText = text.split()
    if len(splitText) > 1:
        firstTwoWords = splitText[0] + ' ' + splitText[1]
        for i in range (0,len(statesList)):
            if firstTwoWords == statesList[i]:
                if i%2 == 0:
                    return firstTwoWords
                else:
                    return statesList[i-1]
        lastTwoWords = splitText[len(splitText)-1] + ' ' + splitText[len(splitText)-2]
        for i in range (0,len(statesList)):
            if lastTwoWords == statesList[i]:
                if i%2 == 0:
                    return lastTwoWords
                else:
                    return statesList[i-1]
        if len(splitText) > 2:
            firstThreeWords = splitText[0] + ' ' + splitText[1]+ ' ' + splitText[2]
            lastThreeWords = splitText[len(splitText)-1] + ' ' + splitText[len(splitText)-2]+ ' ' + splitText[len(splitText)-3]
            for i in range (0,len(statesList)):
                if firstThreeWords == statesList[i]:
                    if i%2 == 0:
                        return firstThreeWords
                    else:
                        return statesList[i-1]
            for i in range (0,len(statesList)):
                if lastThreeWords == statesList[i]:
                    if i%2 == 0:
                        return lastThreeWords
                    else:
                        return statesList[i-1]
        for i in range(0, len(splitText)):
            for j in range (0,len(statesList)):
                if splitText[i] == statesList[j]:
                    if j%2 == 0:
                        return splitText[i]
                    else:
                        return statesList[j-1]
    else:
        for i in range (0,len(statesList)):
            if text == statesList[i]:
                if i%2 == 0:
                    return text
                else:
                    return statesList[i-1]
    return ''

In [18]:
def extractCountry(text):
    if text == '':
        return ''
    if text == 'united states of america':
        return 'united states of america'
    splitText = text.split()
    if len(splitText) > 1:
        firstTwoWords = splitText[0] + ' ' + splitText[1]
        for i in range (0,len(countriesList)):
            if firstTwoWords == countriesList[i]:
                if i<len(countriesList)/2:
                    return countriesList[i+int(len(countriesList)/2)]
        lastTwoWords = splitText[len(splitText)-1] + ' ' + splitText[len(splitText)-2]
        for i in range (0,len(countriesList)):
            if lastTwoWords == countriesList[i]:
                if i<len(countriesList)/2:
                    return countriesList[i+int(len(countriesList)/2)]
        if len(splitText) > 2:
            firstThreeWords = splitText[0] + ' ' + splitText[1]+ ' ' + splitText[2]
            lastThreeWords = splitText[len(splitText)-1] + ' ' + splitText[len(splitText)-2]+ ' ' + splitText[len(splitText)-3]
            for i in range (0,len(countriesList)):
                if firstThreeWords == countriesList[i]:
                    if i<len(countriesList)/2:
                        return countriesList[i+int(len(countriesList)/2)]
            for i in range (0,len(countriesList)):
                if lastThreeWords == countriesList[i]:
                    if i<len(countriesList)/2:
                        return countriesList[i+int(len(countriesList)/2)]
        for i in range(0, len(splitText)):
            for j in range (0,len(countriesList)):
                if splitText[i] == 'usa' or splitText[i] == 'uk':
                    return splitText[i] 
                if splitText[i] == countriesList[j]:
                    if j<len(countriesList)/2:
                        return countriesList[j+int(len(countriesList)/2)]
    else:
        for i in range (0,len(countriesList)):
            if text == 'usa' or text == 'uk':
                    return text 
            if text == countriesList[i]:
                if i<len(countriesList)/2:
                    return countriesList[i+int(len(countriesList)/2)]

    return ''

In [19]:
def incomeCityFeature(text):
    income = incomeDict.get(text)
    return income

In [20]:
def incomeStateFeature(text):
    income = stateIncomeDict.get(text)
    return income

In [21]:
def incomeCountryFeature(text):
    countryIncome = countryIncomeDict.get(text)
    return countryIncome

In [22]:
def incomeEducationFeature(text):
    countryEducation = countryEducationDict.get(text)
    return countryEducation

In [23]:
def stateEducationFeature(text):
    stateEducation = stateEducationDict.get(text)
    return stateEducation

In [24]:
path = '50M_250_1000_1_final_updated_desc.csv'

In [25]:
df = pd.read_csv(path,index_col=None, header=0, lineterminator='\n')

In [26]:
tweetPlaceDF = df['user_location']
tweetPlaceDF = tweetPlaceDF.dropna()
tweetPlaceDF = tweetPlaceDF.apply(strip_links)
tweetPlaceDF = tweetPlaceDF.apply(remove_special_characters)
tweetPlaceDF = tweetPlaceDF.apply(lambda x : x.lower())
tweetPlaceDF = tweetPlaceDF.apply(removeNums)

In [27]:
extractedCityDF = tweetPlaceDF.apply(extractCity)
extractedCityDF = extractedCityDF[extractedCityDF != '']

In [28]:
extractedStateDF = tweetPlaceDF.apply(extractState)
extractedStateDF = extractedStateDF[extractedStateDF != '']

In [29]:
extractCountryDF = tweetPlaceDF.apply(extractCountry)

In [30]:
extractCountryDF = extractCountryDF[extractCountryDF != '']

In [31]:
def stateToCountry(text):
    if text is not None and text != '':
        return 'us'
stateToCountryDF = extractedStateDF.apply(stateToCountry)

In [42]:
combinedCountryDF = pd.DataFrame( np.hstack((stateToCountryDF.values, extractCountryDF.values))  , index=np.hstack((stateToCountryDF.index.values, extractCountryDF.index.values)))

In [49]:
countryFeature = combinedCountryDF[0]
countryFeature = countryFeature[~countryFeature.index.duplicated()]

In [50]:
cityIncomeColumn = extractedCityDF.apply(incomeCityFeature)
stateIncomeColumn = extractedStateDF.apply(incomeStateFeature)
countryIncomeColumn = countryFeature.apply(incomeCountryFeature)
countryEducationColumn = countryFeature.apply(incomeEducationFeature)
stateEducationColumn = extractedStateDF.apply(stateEducationFeature)

In [51]:
df['city'] = extractedCityDF
df['state'] = extractedStateDF
df['country'] = countryFeature
df['income_by_city'] = cityIncomeColumn
df['income_by_state'] = stateIncomeColumn
df['income_by_country'] = countryIncomeColumn
df['education_by_country'] = countryEducationColumn
df['education_by_state'] = stateEducationColumn

In [56]:
print (df['country'].dropna())

20870      ar
20871      ar
20872      ar
20873      ar
20874      ar
20875      ar
20876      ar
20877      ar
20878      ar
20879      ar
20880      ar
20881      ar
20882      ar
20883      ar
20884      ar
20885      ar
20886      ar
20887      ar
20888      ar
20889      ar
20890      ar
20891      ar
20892      ar
20893      ar
20894      ar
20895      ar
20896      ar
20897      ar
20898      ar
20899      ar
           ..
1738962    us
1738963    us
1738964    us
1738965    us
1738966    us
1738967    us
1738968    us
1738969    us
1738970    us
1738971    us
1738972    us
1738973    us
1738974    us
1738975    us
1738976    us
1738977    us
1738978    us
1738979    us
1738980    us
1738981    us
1738982    us
1738983    us
1738984    us
1738985    us
1738986    us
1738987    us
1738988    us
1738989    us
1738990    us
1738991    us
Name: country, Length: 615520, dtype: object


In [ ]:
#%time df.to_csv('50M_250_1000_1_final_updated_desc.csv', index=False)